<a href="https://colab.research.google.com/github/veenqa/BeyondChats-assignment/blob/main/BeyondChats_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import os
os.environ["REDDIT_CLIENT_ID"] = "cSpqJb9vbsmBZnnwjayzpA"
os.environ["REDDIT_CLIENT_SECRET"] = "ecpKWZB5xNzDC91D0gYyaOyzJ8mjYA"
os.environ["REDDIT_USER_AGENT"] = "PersonaGenerator2025/1.0 by Ok-Respect3852"


In [71]:
import os
import re
import sys
import textwrap
from datetime import datetime

import praw
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

REDDIT_CLIENT_ID     = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT    = os.getenv("REDDIT_USER_AGENT")



def extract_username(url_or_user):
    url_or_user = url_or_user.strip().rstrip("/")
    if url_or_user.startswith("u/") or url_or_user.startswith("user/"):
        return url_or_user.split("/", 1)[1]
    m = re.search(r"reddit\.com\/(?:u|user)\/([^\/]+)", url_or_user, re.IGNORECASE)
    if m:
        return m.group(1)
    if re.fullmatch(r"[A-Za-z0-9_-]+", url_or_user):
        return url_or_user
    return None


def initialize_reddit():
    if not all([REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET, REDDIT_USER_AGENT]):
        print("❌ Set Reddit API credentials as environment variables.")
        sys.exit(1)
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
        check_for_async=False
    )
    return reddit


def scrape_user_data(reddit, username):
    print(f"🔍 Scraping Reddit user: u/{username}")
    data = []
    try:
        user = reddit.redditor(username)
        _ = user.id
    except:
        print("❌ Could not access profile.")
        return []

    for c in user.comments.new(limit=100):
        if c.body not in ['[deleted]', '[removed]']:
            data.append(f"Comment in r/{c.subreddit.display_name}: {c.body.strip()}")
    for s in user.submissions.new(limit=50):
        text = s.title + "\n\n" + s.selftext if s.selftext else s.title
        if text and text not in ['[deleted]', '[removed]']:
            data.append(f"Post in r/{s.subreddit.display_name}: {text.strip()}")

    print(f"✅ Scraped {len(data)} total items.")
    return data


def load_hf_model():
    print("🧠 Loading summarization model…")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    return summarizer


def generate_persona(summarizer, texts, username):
    print("🧠 Generating persona…")
    combined = "\n\n".join(texts[:30])
    chunk = combined[:1024]  # Truncate if needed
    summary = summarizer(chunk, max_length=250, min_length=80, do_sample=False)[0]['summary_text']

    persona = f"""Reddit User Persona: u/{username}
Generated using a Hugging Face summarization model.

Summary of Behavior:
{summary.strip()}

Note: This persona was inferred from Reddit posts/comments.
"""
    return persona


def save_persona(username, persona_text):
    filename = f"{username}_persona.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(persona_text)
    return filename


def main():
    print("🔵 Reddit Persona Generator (Hugging Face)")
    print("=" * 60)

    profile = input("Enter Reddit profile URL or username: ").strip()
    username = extract_username(profile)
    if not username:
        print("❌ Invalid Reddit username.")
        sys.exit(1)

    reddit = initialize_reddit()
    data = scrape_user_data(reddit, username)
    if not data:
        print("❌ No content found.")
        sys.exit(1)

    summarizer = load_hf_model()
    persona = generate_persona(summarizer, data, username)
    file = save_persona(username, persona)

    print(f"\n💾 Saved to: {file}")
    print("\n📝 Preview:")
    print("-" * 60)
    print(textwrap.indent(persona[:600], "  "))
    print("✅ Done.")


if __name__ == "__main__":
    main()



🔵 Reddit Persona Generator (Hugging Face)
Enter Reddit profile URL or username: https://www.reddit.com/user/Hungry-Move-6603/
🔍 Scraping Reddit user: u/Hungry-Move-6603
✅ Scraped 12 total items.
🧠 Loading summarization model…


Device set to use cpu


🧠 Generating persona…

💾 Saved to: Hungry-Move-6603_persona.txt

📝 Preview:
------------------------------------------------------------
  Reddit User Persona: u/Hungry-Move-6603
  Generated using a Hugging Face summarization model.

  Summary of Behavior:
  Comment in r/nagpur: I was caught without helmet and license (close to my home). Cops outright wanted to fine me, but a 'common guy' came in and discussed bribe on my behalf with cops . I gave him 200rs (back in 2011) and Cops let me go . He was not a common man. He was their agent and shield. If I or anyone recorded myself handing out money, it would be to a common . man, not cops directly .

  Note: This persona was inferred from Reddit posts/comments.

✅ Done.


In [72]:
import os
import re
import sys
import textwrap
from datetime import datetime

import praw
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


REDDIT_CLIENT_ID     = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT    = os.getenv("REDDIT_USER_AGENT")



def extract_username(url_or_user):
    url_or_user = url_or_user.strip().rstrip("/")
    if url_or_user.startswith("u/") or url_or_user.startswith("user/"):
        return url_or_user.split("/", 1)[1]
    m = re.search(r"reddit\.com\/(?:u|user)\/([^\/]+)", url_or_user, re.IGNORECASE)
    if m:
        return m.group(1)
    if re.fullmatch(r"[A-Za-z0-9_-]+", url_or_user):
        return url_or_user
    return None


def initialize_reddit():
    if not all([REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET, REDDIT_USER_AGENT]):
        print("❌ Set Reddit API credentials as environment variables.")
        sys.exit(1)
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
        check_for_async=False
    )
    return reddit


def scrape_user_data(reddit, username):
    print(f"🔍 Scraping Reddit user: u/{username}")
    data = []
    try:
        user = reddit.redditor(username)
        _ = user.id
    except:
        print("❌ Could not access profile.")
        return []

    for c in user.comments.new(limit=100):
        if c.body not in ['[deleted]', '[removed]']:
            data.append(f"Comment in r/{c.subreddit.display_name}: {c.body.strip()}")
    for s in user.submissions.new(limit=50):
        text = s.title + "\n\n" + s.selftext if s.selftext else s.title
        if text and text not in ['[deleted]', '[removed]']:
            data.append(f"Post in r/{s.subreddit.display_name}: {text.strip()}")

    print(f"✅ Scraped {len(data)} total items.")
    return data


def load_hf_model():
    print("🧠 Loading summarization model…")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    return summarizer


def generate_persona(summarizer, texts, username):
    print("🧠 Generating persona…")
    combined = "\n\n".join(texts[:30])
    chunk = combined[:1024]  # Truncate if needed
    summary = summarizer(chunk, max_length=250, min_length=80, do_sample=False)[0]['summary_text']

    persona = f"""Reddit User Persona: u/{username}
Generated using a Hugging Face summarization model.

Summary of Behavior:
{summary.strip()}

Note: This persona was inferred from Reddit posts/comments.
"""
    return persona


def save_persona(username, persona_text):
    filename = f"{username}_persona.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(persona_text)
    return filename


def main():
    print("🔵 Reddit Persona Generator (Hugging Face)")
    print("=" * 60)

    profile = input("Enter Reddit profile URL or username: ").strip()
    username = extract_username(profile)
    if not username:
        print("❌ Invalid Reddit username.")
        sys.exit(1)

    reddit = initialize_reddit()
    data = scrape_user_data(reddit, username)
    if not data:
        print("❌ No content found.")
        sys.exit(1)

    summarizer = load_hf_model()
    persona = generate_persona(summarizer, data, username)
    file = save_persona(username, persona)

    print(f"\n💾 Saved to: {file}")
    print("\n📝 Preview:")
    print("-" * 60)
    print(textwrap.indent(persona[:600], "  "))
    print("✅ Done.")


if __name__ == "__main__":
    main()


🔵 Reddit Persona Generator (Hugging Face)
Enter Reddit profile URL or username: https://www.reddit.com/user/gallowboob/
🔍 Scraping Reddit user: u/gallowboob
✅ Scraped 150 total items.
🧠 Loading summarization model…


Device set to use cpu


🧠 Generating persona…

💾 Saved to: gallowboob_persona.txt

📝 Preview:
------------------------------------------------------------
  Reddit User Persona: u/gallowboob
  Generated using a Hugging Face summarization model.

  Summary of Behavior:
  It would be too noisy to go ham on this account. I could find a trusted surrogate user to feed content through and not bother making an alt (i'm jk... unless?) I didn’t think I could love anything this much. She’s unreal alright. I’m happy to see this comment! I've always been  a fan of reddit. It looks like SOMETHING is finally happening. Maybe I never left .

  Note: This persona was inferred from Reddit posts/comments.

✅ Done.
